In [1]:
import os
import sys
import re
project_path = re.match('/.+interview-practice/', os.getcwd())[0]
sys.path.append(project_path)

from collections import deque
from functions.structs.tree import balanced_tree_from_sorted_array
from functions.formatting.list_tools import peek
from functions.custom_errors import MethodNotFoundError

In [2]:
class Node:
    def __init__(self, data=None):
        self.data = data
        self.left = None
        self.right = None

class BinaryTree:
    """Make sure to only add Node objects to the Binary Tree
    
    Example 1: Using insert only
    source: https://www.tutorialspoint.com/python_data_structure/python_tree_traversal_algorithms.htm
           27
         /    \
       14      35
      /  \    /  \
    10   19  31  42
    
    tree = BinaryTree(Node(27))
    tree.insert(Node(14))
    tree.insert(Node(35))
    tree.insert(Node(10))
    tree.insert(Node(19))
    tree.insert(Node(31))
    tree.insert(Node(42))
    
    Example 2: Impossible with insert only
    source: https://www.section.io/engineering-education/binary-tree-data-structure-python/

           10
         /    \
       34      89
      /  \    /  \
    20   45  56  54
    
    tree = BinaryTree(Node(10))
    tree.root.left = Node(34)
    tree.root.right = Node(89)
    tree.root.left.left = Node(20)
    tree.root.left.right = Node(45)
    tree.root.right.left = Node(56)
    tree.root.right.right = Node(54)
    """
    
    def __init__(self, root=None):
        self.root = root
        self.current = self.root
    
    def get_current(self):
        return self.current.data
    
    def move_left(self):
        self.current = self.current.left
        return self.current.data
    
    def move_right(self):
        self.current = self.current.right
        return self.current.data
    
    def reset(self):
        self.current = self.root
        return self.root.data
        
    def __insert(self, node, new_node):
        """Traverse down the tree, insert at first open position
        Inserts in order
        """
        
        if new_node.data < node.data:
            if node.left is None:
                node.left = new_node
            else:
                self.__insert(node.left, new_node)

        elif new_node.data > node.data:
            if node.right is None:
                node.right = new_node
            else:
                self.__insert(node.right, new_node)
            
    def insert(self, new_node):
        """level order insert
        """
        if self.root is None:
            self.root = new_node
        else:
            self.__insert(self.root, new_node)
           
    def count_leaf_nodes(self, node='default'):
        """recursive count
        """
        if node=='default':
            node = self.root
        
        if node is None:
            return 0
        if(node.left is None and node.right is None):
            return 1
        else: 
            return self.count_leaf_nodes(node.left) + self.count_leaf_nodes(node.right)
            
    def iterative_levelorder(self, node='default'):
        """traverse iteratively
        add current level elements to stack and pop them to return
        """
        
        stack = deque()
        stack.append(self.root)

        while stack:
            node = stack.popleft()
            yield node

            if node.left is not None:
                stack.append(node.left)

            if node.right is not None:
                stack.append(node.right)
    
    def recursive_inorder(self, node='default'):
        """traverse recursively
        left -> root -> right
        """
        
        if node=='default':
            node = self.root

        if node.left:
            yield from self.recursive_inorder(node.left)
            
        yield node
        
        if node.right:
            yield from self.recursive_inorder(node.right)

    def recursive_rev_inorder(self, node='default'):
        """traverse recursively
        right -> root -> left
        """
        
        if node=='default':
            node = self.root
        
        if node.right:
            yield from self.recursive_rev_inorder(node.right)
            
        yield node
        
        if node.left:
            yield from self.recursive_rev_inorder(node.left)
            
    def recursive_postorder(self, node='default'):
        """traverse recursively
        left -> right -> root
        """
        
        if node=='default':
            node = self.root
        
        if node.left:
            yield from self.recursive_postorder(node.left)
        
        if node.right:
            yield from self.recursive_postorder(node.right)
            
        yield node
        
    def recursive_preorder(self, node='default'):
        """traverse recursively
        root -> left -> right
        """

        if node=='default':
            node = self.root
            
        yield node
        
        if node.left:
            yield from self.recursive_preorder(node.left)
        
        if node.right:
            yield from self.recursive_preorder(node.right)
    
    def iterative_inorder(self, node='default'):
        """See: https://www.geeksforgeeks.org/inorder-tree-traversal-without-recursion/
        
        1) Create an empty stack.
        2) Initialize current node as root
        3) Push the current node to S and set current = current->left until current is NULL
        4) If current is NULL and stack is not empty then 
             a) Pop the top item from stack.
             b) Print the popped item, set current = popped_item->right 
             c) Go to step 3.
        5) If current is NULL and stack is empty then we are done.
        
        Example 1:
        add 27, add 14, add 10, pop 10, return 10, node.right=None
        pop 14, return 14, node.right=19
        add 19, pop 19, return 19, node.right=None
        pop 27, return 27, node.right=35
        add 35, add 31, pop 31, return 31, node.right=None
        pop 35, return 35, node.right=42
        add 42, pop 42, return 42, node.right=None
        """
        
        if node=='default':
            node = self.root
            
        stack = deque()
        
        while True:
            # during first loop, adds all left nodes to stack
            # during subsequent loops, may add if node.right exists
            while node is not None:
                stack.append(node)
                node = node.left
            
            if stack:
                node = stack.pop()
                yield node
                node = node.right  # adds node.right to stack if it exists
                
            else:
                break
    
    def iterative_preorder(self, node='default'):
        
        if node=='default':
            node = self.root
            
        stack = deque()
        stack.append(node)
        
        while stack:
            node = stack.pop()
            yield node
            
            if node.right:
                stack.append(node.right)
                
            if node.left:
                stack.append(node.left)
        
    def iterative_postorder(self, node='default'):
        
        if node=='default':
            node = self.root
            
        stack = deque()
        
        while True:
            while node:
                if node.right is not None: 
                    stack.append(node.right)
                    
                stack.append(node)
                node = node.left
            
            node = stack.pop()
            
            if node.right is not None and peek(stack) == node.right:
                stack.pop()
                stack.append(node)
                node = node.right
                
            else:
                yield node
                node = None
                
            if not stack:
                break            
            
    def print(self, method='recursive', order='inorder'):
        """Valid options:
        order: ['inorder', 'preorder', 'postorder']
        method: ['recursive', 'iterative']
        """
        
        traverse = {'recursive': {'inorder': self.recursive_inorder(),
                                  'preorder': self.recursive_preorder(),
                                  'postorder': self.recursive_postorder(),},
                    'iterative': {'inorder': self.iterative_inorder(),
                                  'preorder': self.iterative_preorder(),
                                  'postorder': self.iterative_postorder(),
                                  'levelorder': self.iterative_levelorder(),}}
        
        if traverse.get(method, {}).get(order) is None:
            raise MethodNotFoundError(f"Please check your inputs: order='{order}', method='{method}'")

        for node in traverse[method][order]:
            print(node.data)
            
    def print_leaf_nodes(self, order='left_to_right'):
        if order=='left_to_right':
            for node in self.recursive_inorder():
                if (node.left is None and node.right is None):
                    print(node.data)
                    
        elif order=='right_to_left':
            for node in self.recursive_rev_inorder():
                if (node.left is None and node.right is None):
                    print(node.data)
                    
        else:
            raise MethodNotFoundError(f"Please check your input: order='{order}'")
        
    def search(self, data, order='levelorder'):
        """level order search"""
        
        traverse = {'levelorder': self.iterative_levelorder(),
                    'inorder': self.recursive_inorder(),
                    'preorder': self.recursive_preorder(),
                    'postorder': self.recursive_postorder()}
        
        if traverse.get(order) is None:
            raise MethodNotFoundError(f"Please check your inputs: method='{method}'")
        
        for node in traverse[order]:
            if node.data == data:
                return node
        return print("Not found")

In [3]:
# to do:
# delete
# balancing trees

In [4]:
tree = BinaryTree(Node(27))
tree.insert(Node(14))
tree.insert(Node(35))
tree.insert(Node(10))
tree.insert(Node(19))
tree.insert(Node(31))
tree.insert(Node(42))

In [5]:
tree.print_leaf_nodes()

10
19
31
42


In [6]:
# balance an unbalanced tree:

unbalanced_tree = BinaryTree(Node(10))
unbalanced_tree.root.left = Node(34)
unbalanced_tree.root.right = Node(89)
unbalanced_tree.root.left.left = Node(20)
unbalanced_tree.root.left.right = Node(45)
unbalanced_tree.root.right.left = Node(56)
unbalanced_tree.root.right.right = Node(54)

In [7]:
unbalanced_tree.print()

20
34
45
10
56
89
54
